In [170]:
import gspread
from google.oauth2.service_account import Credentials

In [171]:
scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = Credentials.from_service_account_file("credentials.json", scopes=scopes)
client = gspread.authorize(creds)

In [172]:
sheet_id = "1-7XFGkuXXEC9HcnVubukZKdo5tqrieVLIga9L6bPKxk"
sheet = client.open_by_key(sheet_id).sheet1

# Fetch Live Data

In [173]:
import requests

In [174]:
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",          # Get prices in USD
        "order": "market_cap_desc",    # Order by market capitalization
        "per_page": 50,                # Number of cryptocurrencies to fetch
        "page": 1,                     # Fetch the first page of results
        "sparkline": False             # Exclude sparkline data
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: Unable to fetch data (Status Code: {response.status_code})")
        return None

In [175]:
crypto_data = fetch_crypto_data()
if crypto_data:
    print("Top 50 Cryptocurrencies:")
    for i, coin in enumerate(crypto_data, start=1):
        print(f"{i}. {coin['name']} ({coin['symbol'].upper()}) - ${coin['current_price']} {coin['market_cap']} {coin['total_volume']} {coin['price_change_percentage_24h']}")

Top 50 Cryptocurrencies:
1. Bitcoin (BTC) - $93823 1858837258854 51174679602 -0.60705
2. Ethereum (ETH) - $3254.19 392030975556 29061935144 -2.53401
3. Tether (USDT) - $0.999901 137431825236 100554464845 0.03835
4. XRP (XRP) - $2.29 131622983145 5082814875 -2.8578
5. BNB (BNB) - $690.01 100665547360 1049009113 -1.35732
6. Solana (SOL) - $189.12 91459464158 4576887586 -3.09192
7. Dogecoin (DOGE) - $0.327608 48340152899 3612640670 -3.59201
8. USDC (USDC) - $0.999987 45412954612 6194739018 -0.00073
9. Cardano (ADA) - $0.918796 32946272493 1241103784 -2.205
10. Lido Staked Ether (STETH) - $3252.18 31303473596 66796009 -2.36157
11. TRON (TRX) - $0.240971 20772061339 1154422839 -2.67359
12. Avalanche (AVAX) - $36.86 15130091326 408922647 -0.78216
13. Sui (SUI) - $4.86 14622623920 1918595484 1.34102
14. Wrapped stETH (WSTETH) - $3870.47 13654319627 58635119 -2.28366
15. Toncoin (TON) - $5.21 13241808919 233191006 -1.61667
16. Chainlink (LINK) - $19.95 12590016558 858062008 -1.76941
17. Shiba 

# Data Analysis

In [176]:
def analyze_crypto_data(data):
    
    # Identify the top 5 cryptocurrencies by market capitalization
    top_5_by_market_cap = sorted(data, key=lambda x: x['market_cap'], reverse=True)[:5]

    # Calculate the average price of the top 50 cryptocurrencies
    average_price = sum(coin['current_price'] for coin in data) / len(data)

    # Find the highest and lowest percentage change in 24 hours
    highest_change = max(data, key=lambda x: x['price_change_percentage_24h'])
    lowest_change = min(data, key=lambda x: x['price_change_percentage_24h'])

    # Display the analysis results
    print("\n--- Data Analysis ---")
    print("\nTop 5 Cryptocurrencies by Market Cap:")
    i=1
    for coin in top_5_by_market_cap:
        print(f"{i}. {coin['name']} ({coin['symbol'].upper()}) - Market Cap: ${coin['market_cap']:,}")
        i += 1

    print(f"\nAverage Price of Top 50 Cryptocurrencies: ${average_price:.2f}")
    print(f"\nHighest 24h % Change: {highest_change['name']} ({highest_change['price_change_percentage_24h']:.2f}%)")
    print(f"Lowest 24h % Change: {lowest_change['name']} ({lowest_change['price_change_percentage_24h']:.2f}%)")

In [177]:
analyze_crypto_data(crypto_data)


--- Data Analysis ---

Top 5 Cryptocurrencies by Market Cap:
1. Bitcoin (BTC) - Market Cap: $1,858,837,258,854
2. Ethereum (ETH) - Market Cap: $392,030,975,556
3. Tether (USDT) - Market Cap: $137,431,825,236
4. XRP (XRP) - Market Cap: $131,622,983,145
5. BNB (BNB) - Market Cap: $100,665,547,360

Average Price of Top 50 Cryptocurrencies: $4142.67

Highest 24h % Change: Bitget Token (4.49%)
Lowest 24h % Change: Bittensor (-6.51%)


# Live-Running Excel Sheet

In [178]:
import openpyxl
import time
from openpyxl.styles import PatternFill
import os
from fpdf import FPDF

In [179]:
def update_google_sheet(data):
   
    headers = ['Cryptocurrency Name', 'Symbol', 'Current Price (USD)', 'Market Capitalization',
               '24-hour Trading Volume', 'Price Change (24h, %)']
    
    existing_data = sheet.get_all_values()

    if len(existing_data) == 0:
        sheet.append_row(headers)
        existing_data = [headers]

    for coin in data:
        updated = False
        for i, row in enumerate(existing_data[1:], start=1):
            if row[1].upper() == coin['symbol'].upper():
                
                existing_data[i] = [
                    coin['name'],
                    coin['symbol'].upper(),
                    coin['current_price'],
                    coin['market_cap'],
                    coin['total_volume'],
                    coin['price_change_percentage_24h']
                ]
                updated = True
                break

        if not updated:
            existing_data.append([
                coin['name'],
                coin['symbol'].upper(),
                coin['current_price'],
                coin['market_cap'],
                coin['total_volume'],
                coin['price_change_percentage_24h']
            ])

    sheet.clear()
    sheet.append_rows(existing_data, value_input_option="USER_ENTERED")

In [180]:
def update_excel_sheet(data):
    file_path = 'crypto_data.xlsx'
    
    if os.path.exists(file_path):
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active
    else:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet['A1'] = 'Cryptocurrency Name'
        sheet['B1'] = 'Symbol'
        sheet['C1'] = 'Current Price (USD)'
        sheet['D1'] = 'Market Capitalization'
        sheet['E1'] = '24-hour Trading Volume'
        sheet['F1'] = 'Price Change (24h, %)'

    # Create fill styles for row highlighting
    red_fill = PatternFill(start_color='FFCCCB', end_color='FFCCCB', fill_type='solid')  # Red for decrease
    green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')  # Green for increase

    for coin in data:
        # Check if the coin already exists in the sheet by its name (column A)
        existing_row = None
        for row in range(2, sheet.max_row + 1):  # Skip header row
            if sheet[f'A{row}'].value == coin['name']:
                existing_row = row
                break

        if existing_row:
            # Update existing row
            sheet[f'C{existing_row}'] = coin['current_price']
            sheet[f'D{existing_row}'] = coin['market_cap']
            sheet[f'E{existing_row}'] = coin['total_volume']
            sheet[f'F{existing_row}'] = coin['price_change_percentage_24h']

            price_change = coin['price_change_percentage_24h']
            
            # Apply color fill based on price change
            fill = green_fill if price_change >= 0 else red_fill
            for col in range(1, 7):  # Columns A to F
                sheet.cell(row=existing_row, column=col).fill = fill
        else:
            # Add new row for the cryptocurrency
            row = sheet.max_row + 1
            sheet[f'A{row}'] = coin['name']
            sheet[f'B{row}'] = coin['symbol'].upper()
            sheet[f'C{row}'] = coin['current_price']
            sheet[f'D{row}'] = coin['market_cap']
            sheet[f'E{row}'] = coin['total_volume']
            sheet[f'F{row}'] = coin['price_change_percentage_24h']

            price_change = coin['price_change_percentage_24h']
            
            # Apply color fill based on price change
            fill = green_fill if price_change >= 0 else red_fill
            for col in range(1, 7):  # Columns A to F
                sheet.cell(row=row, column=col).fill = fill

    workbook.save(file_path)

In [181]:
def generate_analysis_report(data):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    pdf.set_font('Arial', 'B', 16)
    pdf.cell(200, 10, txt="Cryptocurrency Market Analysis", ln=True, align='C')

    pdf.set_font('Arial', '', 12)
    pdf.ln(10)
    pdf.cell(200, 10, txt="Key Insights and Analysis", ln=True)

    pdf.ln(10)
    pdf.set_font('Arial', '', 10)
    pdf.cell(30, 10, 'Name', border=1)
    pdf.cell(30, 10, 'Symbol', border=1)
    pdf.cell(50, 10, 'Current Price (USD)', border=1)
    pdf.cell(50, 10, '24h Change (%)', border=1)
    pdf.ln(10)

    for coin in data:
        pdf.cell(30, 10, coin['name'], border=1)
        pdf.cell(30, 10, coin['symbol'], border=1)
        pdf.cell(50, 10, f"${coin['current_price']}", border=1)
        pdf.cell(50, 10, f"{coin['price_change_percentage_24h']}%", border=1)
        pdf.ln(10)

    pdf.output("analysis_report.pdf")

In [182]:
def run_live_updates():
    while True:
        crypto_data = fetch_crypto_data()
        if crypto_data:
            #update_excel_sheet(crypto_data)
            update_google_sheet(crypto_data)
            #generate_analysis_report(crypto_data)
            
        time.sleep(300)

In [183]:
run_live_updates()

KeyboardInterrupt: 